Description: Detect edge & evaluate FOM of proposed method with various noise level

Python version: 3.9

In [1]:
# Link Google colab notebook to your personal google drive account.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install packages
!pip install ruptures
!pip install skan
!pip install aicsimageio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 121.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 KB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.8 MB/s eta 0:00:00
  Attempting uninstall: Pygments
    Found existing installation: Pygments 2.6.1
    Uninstalling Pygments-2.6.1:
      Successfully uninstalled Pygments-2.6.1
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    U

In [3]:
# %% Import libraries
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io  # v.0.18.1
from skimage import filters
from skimage.morphology import disk
from skimage import io
from pathlib import Path
from os import sep
from cv2 import morphologyEx, MORPH_OPEN
import sys
from aicsimageio.writers.ome_tiff_writer import OmeTiffWriter
from aicsimageio import AICSImage

In [4]:
# Import original module
project_path = f"{sep}content{sep}drive{sep}My Drive{sep}my_project{sep}"
sys.path.append(project_path)

from cpseg_module.cpseg_functions_colab import ak_rotate_edge_overlay, postprocess, connect_ends

In [5]:
# Input image file names in /content/drive/my_project
imFile = "test_data_4d.tif"

In [6]:
# %% Parameters
# Image file
rootfile = f"{sep}content{sep}drive{sep}My Drive{sep}my_project{sep}"
# Get image file name
imFilename = f"{rootfile}{imFile}"              # Image file path                                    
se_size = 200                                   # Background subtraction
medianValue = [5, 5]#[30, 30]                   # Median filter
y_interval = 2  # 5                             # Sparse scan along y-axis, pixel
rotation_angle = 4 #3                           # 0: 45 degree, 3: semicircle
                                                # 1: 30 degree, 4: semicircle
                                                # 2: 15 degree, 5: semicircle
changepoint_algo = 1                            # 0: number known, 1: number not known
numBorder = 4                                   # Number of change points,
                                                # if changepoint_algo = 1, use PELT for panelized detection
penalty_value = 100000                          # Penalty cost when numBorder = 0
numlayer = 1                                    # 0: single <- avoid in this workflow
                                                # 1: overlay multiple
model = "linear"                                # "rbf", "linear", "cosine"
smallNoiseRemov = 1                             # 0:No noise removal
                                                # 1:Connect border, then remove small noises
                                                # 2:Remove small noisess first, then connect
senoise = y_interval * 2                        # Structural element size of morphology opening:default:5
neib = 25                                       # Remove small objects less than neib(pixel) default:100
select_biggest = 0                              # 0: No selection; 1: select biggest
nskeletonize = 1                                # 0: No skeletonization,
                                                # 1: with skeletonization
postProcessing =    1                           # 0: No
                                                # 1: Yes
min_j2e_size = 100                              # Remove branch smaller than min_j2e_size                                                  #(pixels), default 1000000 pix
connect_gap = 1                                 # 0: No
                                                # 1: Yes
implot =            0                           # 0: No image
                                                # 1: Plot image
save_data =         1                           # 0: Do not save data
                                                # 1: Save data

In [8]:
#%% Processing

# Load info
I0 = AICSImage(imFilename) # Load image

# Create edge container
shape4d = list(I0.shape)
shape4d.pop(1)
BW_proposed_pp = np.zeros(shape4d, dtype='int8')

# Create parameter lists
Ts = np.arange(I0.dims.T)
Cs = np.arange(I0.dims.C)
Zs = np.arange(I0.dims.Z)
TCZ_list = [(t,c,z) for t in Ts for c in Cs for z in Zs]

for t, c, z in TCZ_list:
    print(F'Time# is {t}, z-slice# is {z}.')
    lazy_Iori = I0.get_image_dask_data("YX", T=t, C=c, Z=z) # returns out-of-memory 4D dask array
    Iori = lazy_Iori.compute() # returns in-memory 4D numpy array

    # Pre-processing
    se_imopen = disk(se_size)                                           # structrul element for morph open
    background = morphologyEx(Iori, MORPH_OPEN, se_imopen)            # opening
    I_bg = Iori - background                                            # Subtract background
    Ien = filters.median(I_bg, disk(medianValue[0]))                  # Smoothing filter

    # Edge detection
    def proposed():
        BW = ak_rotate_edge_overlay\
            (Ien,numlayer,rotation_angle,numBorder,model,y_interval,penalty_value, changepoint_algo)
        print('Edge detected.')

        # Post-processing
        if postProcessing == 0:
            BWlast = np.copy(BW)
        elif postProcessing == 1:
            BWlast = postprocess(BW, min_j2e_size)
            if connect_gap == 1:
                BWlast = connect_ends(BWlast, min_j2e_size)

        #BW_proposed_pp[iz, :, :] = BWlast
        BW_proposed_pp[t, z, :, :] = BWlast

        # Plot images
        if implot == 1:
            fig, ax = plt.subplots(nrows=3, ncols=1, tight_layout=False, figsize=(10, 10), sharex=True, sharey=True)
            ax[0].imshow(Iori, cmap='gray')
            ax[1].imshow(BW, cmap='gray_r')
            ax[2].imshow(BWlast, cmap='gray_r')

            for irows in np.arange(3):
                ax[irows].set_xticks([])
                ax[irows].set_yticks([])

            plt.show()
    proposed()

Time# is 0, z-slice# is 0.
Edge detected.
prune repeat: 1
prune repeat: 2
Time# is 0, z-slice# is 1.
Edge detected.
prune repeat: 1
prune repeat: 2
prune repeat: 3
Time# is 0, z-slice# is 2.
Edge detected.
prune repeat: 1
prune repeat: 2
Time# is 1, z-slice# is 0.
Edge detected.
prune repeat: 1
prune repeat: 2
Time# is 1, z-slice# is 1.
Edge detected.
prune repeat: 1
prune repeat: 2
prune repeat: 3
Time# is 1, z-slice# is 2.
Edge detected.
prune repeat: 1
prune repeat: 2
prune repeat: 3
Time# is 2, z-slice# is 0.
Edge detected.
prune repeat: 1
prune repeat: 2
prune repeat: 3
Time# is 2, z-slice# is 1.
Edge detected.
prune repeat: 1
prune repeat: 2
prune repeat: 3
Time# is 2, z-slice# is 2.
Edge detected.
prune repeat: 1


In [9]:
# Save edge
result_path = f"{project_path}result{sep}"
save_file = f"{result_path}edge_stack.tif"
OmeTiffWriter.save(BW_proposed_pp, save_file, dim_order="TZYX")

/usr/local/lib/python3.8/dist-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
